# NYC Open Data
https://opendata.cityofnewyork.us/

NYC Open Data is a polatform to access open data produced and used by NY City government. 

One can download the data in two ways:
    + Using the website
    + Using the Socrata Open Data API




## Using the Website

From now on, let's focus on the 311 Service Requests from 2010 to Present: 

https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9

One can download the CSV file with all the historical data from 2010 to Present. 

**Pro's**: You get all the data at once

**Cons**: The file is heavy (>20 GB) and might be hard to open in a laptop without enough RAM.

![Drag Racing](figs/311_download_screenshot.png)

## Using the Socrata Open Data API

I fully recommend this approach since it allows you to access the part of information you need. You also save time since you can download the information in different runs.

For this to work, you need to obtain an app token. First, create a free account at:

https://data.cityofnewyork.us/login

Please remember the email and password used, because you will need them later.

Once you log in, you click on 'Developer Settings' -> 'Create New App Token': 

![AppToken1](figs/appToken1.png)

Now we are ready to download the data

### Client Connection

We can create a client connection using using the sodapy library. You will need: 
    + token
    + username (the email address)
    + password

For more details about sodapy library check: https://pypi.org/project/sodapy/

In [11]:
# !pip install sodapy
import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# Example authenticated client (needed for non-public datasets):
MyAppToken = 'YOUR_TOKEN'
client = Socrata("data.cityofnewyork.us",
                  MyAppToken,
                  username="YOUR_USERNAME",
                  password="YOUR_PASSWORD")

One convenient feature of using the Socrate Open Data API, is that you can do queries specifying the condition that you need for your data, just like a 'WHERE (CONDITION)' statement within a SQL query. This is implemented via the Socrates Query Language 'SoQL'.

For more details regarding possible queries, please check: https://dev.socrata.com/docs/queries/



### Download the first 100 observation in January-2020

In [8]:
query_condition = "created_date between '2020-01-01' and '2020-01-31'"
results = client.get("erm2-nwe9", where=query_condition, limit=100)
df = pd.DataFrame.from_records(results) 
df.head()


,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,x_coordinate_state_plane,y_coordinate_state_plane,open_data_channel_type,park_facility_name,park_borough,latitude,longitude,location,taxi_pick_up_location,facility_type
0,45289558,2020-01-01T00:00:00.000,2020-01-15T00:00:01.000,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,3 or More,Other (Explain Below),11215,625 UNION STREET,...,988567,186277,ONLINE,Unspecified,BROOKLYN,40.677963041857886,-73.98443609121443,"{'latitude': '40.677963041857886', 'longitude'...",NaN,NaN
1,45288728,2020-01-01T00:00:00.000,2020-01-02T00:00:01.000,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,1 or 2,Restaurant/Bar/Deli/Bakery,11225,985 NOSTRAND AVENUE,...,997849,181347,ONLINE,Unspecified,BROOKLYN,40.66442190467239,-73.95098201556382,"{'latitude': '40.66442190467239', 'longitude':...",NaN,NaN
2,45288240,2020-01-01T00:00:00.000,2020-01-02T00:00:01.000,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,1 or 2,Restaurant/Bar/Deli/Bakery,11385,1717 CORNELIA STREET,...,1010470,194453,ONLINE,Unspecified,QUEENS,40.700366489799876,-73.90543829006366,"{'latitude': '40.700366489799876', 'longitude'...",NaN,NaN
3,45287907,2020-01-01T00:00:00.000,2020-01-02T00:00:01.000,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,3 or More,Restaurant/Bar/Deli/Bakery,11214,1602 SHORE PARKWAY,...,984202,156289,ONLINE,Unspecified,BROOKLYN,40.59565343138651,-74.00017283917487,"{'latitude': '40.59565343138651', 'longitude':...",NaN,NaN
4,45285651,2020-01-01T00:00:00.000,2020-01-02T00:00:01.000,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,1 or 2,Restaurant/Bar/Deli/Bakery,10458,2701 DECATUR AVENUE,...,1015012,254391,PHONE,Unspecified,BRONX,40.86486556770799,-73.88878325729915,"{'latitude': '40.86486556770799', 'longitude':...",NaN,NaN


To get the first and last days of every month in a given year, we can use the function `first_and_last_date`

In [12]:
import calendar
import datetime
import collections
def first_and_last_date(year):
    # year:= integer value
    # returns: dictionary with first and last dates for a given year
    first_and_last_days = collections.defaultdict(list)
    for i in range(1,13):
        _, num_days = calendar.monthrange(year, i)
        first_day = datetime.date(year, i, 1).strftime('%Y-%m-%d')
        last_day = datetime.date(year, i, num_days).strftime('%Y-%m-%d')
        first_and_last_days[i] = [first_day, last_day ]
    return(first_and_last_days)

months_days = first_and_last_date(year = 2020)
months_days

defaultdict(list,
            {1: ['2020-01-01', '2020-01-31'],
             2: ['2020-02-01', '2020-02-29'],
             3: ['2020-03-01', '2020-03-31'],
             4: ['2020-04-01', '2020-04-30'],
             5: ['2020-05-01', '2020-05-31'],
             6: ['2020-06-01', '2020-06-30'],
             7: ['2020-07-01', '2020-07-31'],
             8: ['2020-08-01', '2020-08-31'],
             9: ['2020-09-01', '2020-09-30'],
             10: ['2020-10-01', '2020-10-31'],
             11: ['2020-11-01', '2020-11-30'],
             12: ['2020-12-01', '2020-12-31']})

**Download single file for a month**: In this example we download the data for January 2020. Notice that the function queries the data in batches of 3000 observations. We store all the queries into a list of dataframes and then we merge them all.

In [14]:
from os import path

# Input
year = 2020
month = 1

# Code
df_list = []
lim = 3000
i = 0
offset = 0
months_days = first_and_last_date(year = year)
filename = ''.join(['%i-' % year, str(month).zfill(2),'.csv'])
while(True):
    if path.exists(filename): break
    where_cond = ''.join(["created_date between '", months_days[month][0], "' and '",months_days[month][1], "'"])
    results = client.get("erm2-nwe9", 
                         where = where_cond, 
                         limit = lim, offset = lim*i)
    if not results: break
    df = pd.DataFrame.from_records(results)
    df_list.append(df)
    i += 1
df = pd.concat(df_list, axis=0, sort=True)
df.to_csv(filename)



In [15]:
df.shape

(162301, 41)

In [16]:
df.head(10)

,address_type,agency,agency_name,bbl,borough,bridge_highway_direction,bridge_highway_name,bridge_highway_segment,city,closed_date,...,resolution_description,road_ramp,status,street_name,taxi_company_borough,taxi_pick_up_location,unique_key,vehicle_type,x_coordinate_state_plane,y_coordinate_state_plane
0,ADDRESS,DOHMH,Department of Health and Mental Hygiene,3004340016,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2020-01-15T00:00:01.000,...,The Health Departmentâs Office of Environmen...,NaN,Closed,UNION STREET,NaN,NaN,45289558,NaN,988567,186277
1,ADDRESS,DOHMH,Department of Health and Mental Hygiene,3013090001,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2020-01-02T00:00:01.000,...,The Department of Health and Mental Hygiene ha...,NaN,Closed,NOSTRAND AVENUE,NaN,NaN,45288728,NaN,997849,181347
2,ADDRESS,DOHMH,Department of Health and Mental Hygiene,4034610052,QUEENS,NaN,NaN,NaN,Ridgewood,2020-01-02T00:00:01.000,...,The Department of Health and Mental Hygiene ha...,NaN,Closed,CORNELIA STREET,NaN,NaN,45288240,NaN,1010470,194453
3,ADDRESS,DOHMH,Department of Health and Mental Hygiene,3064910050,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2020-01-02T00:00:01.000,...,The Department of Health and Mental Hygiene ha...,NaN,Closed,SHORE PARKWAY,NaN,NaN,45287907,NaN,984202,156289
4,ADDRESS,DOHMH,Department of Health and Mental Hygiene,2032830055,BRONX,NaN,NaN,NaN,BRONX,2020-01-02T00:00:01.000,...,The Department of Health and Mental Hygiene ha...,NaN,Closed,DECATUR AVENUE,NaN,NaN,45285651,NaN,1015012,254391
5,ADDRESS,DOHMH,Department of Health and Mental Hygiene,3073620008,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2020-01-10T00:00:01.000,...,The Health Departmentâs Office of Environmen...,NaN,Closed,NOSTRAND AVENUE,NaN,NaN,45285347,NaN,1000400,157925
6,ADDRESS,DOHMH,Department of Health and Mental Hygiene,4089330077,QUEENS,NaN,NaN,NaN,Woodhaven,2020-01-02T09:39:33.000,...,The Department of Health and Mental Hygiene ha...,NaN,Closed,JAMAICA AVENUE,NaN,NaN,45289555,NaN,1025018,191916
7,ADDRESS,DOHMH,Department of Health and Mental Hygiene,NaN,BRONX,NaN,NaN,NaN,BRONX,2020-01-02T09:51:29.000,...,The Department of Health and Mental Hygiene ha...,NaN,Closed,EAST 149 STREET,NaN,NaN,45288120,NaN,1009724,235487
8,ADDRESS,DOHMH,Department of Health and Mental Hygiene,NaN,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2020-01-02T09:50:09.000,...,The Health Departmentâs Office of Environmen...,NaN,Closed,CHURCH AVENUE,NaN,NaN,45285821,NaN,1005466,177022
9,NaN,NYPD,New York City Police Department,3052300069,BROOKLYN,NaN,NaN,NaN,BROOKLYN,2020-01-01T03:07:10.000,...,The Police Department issued a summons in resp...,NaN,Closed,EAST 28 STREET,NaN,NaN,45282225,NaN,998097,171717
